# **Simulador de perfis de respondentes questionário MMBN-CAS**

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import time
import openai

# Specify the path to your .env file
dotenv_path = "/mnt/4d4f90e5-f220-481e-8701-f0a546491c35/arquivos/projetos/.envpaulo"

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

# Access and store the environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
model = 'gpt-4.1-nano'


In [ ]:
#%pip install -r requirements.txt